In [1]:
# import records
# import sqlite3
# from urllib.parse import urljoin, urldefrag
import requests
from bs4 import BeautifulSoup
import re
from joblib import Parallel, delayed
import pickle
import csv
import MeCab as mc
import numpy as np

# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree as tr

# import matplotlib.pyplot as plt
import random
from gensim.models import word2vec, LdaModel
from gensim import corpora
import gensim


In [ ]:
restaurant_list = []
for page in range(1,51):
    url = f'https://tabelog.com/rstLst/steak/{page}/?Srt=D&SrtT=rt&sort_mode=1&sk=%E3%82%B9%E3%83%86%E3%83%BC%E3%82%AD&svt=1900&svps=2&select_sort_flg=1'
    html_soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    for item in html_soup.find_all():
        
    restaurant_list.append(BeautifulSoup(requests.get(url).text, 'html.parser').find_all('a', class_='list-rst__rst-name-target cpy-rst-name'))

In [5]:
links = []
for restaurants in restaurant_list:
    for restaurant in restaurants:
        links.append(restaurant.get('href'))

In [8]:
tail = 'dtlrvwlst/?smp=0&lc=2&PG='
rev_results = []

# func = lambda review: [i.text for i in BeautifulSoup(requests.get('https://tabelog.com' + review).text, 'html.parser').find_all('div',class_='rvw-item__rvw-comment')]
def get_review(review):
    try:
        res = requests.get('https://tabelog.com' + review, timeout=5)
        return [i.text for i in BeautifulSoup(res.text, 'html.parser').find_all('div',class_='rvw-item__rvw-comment')]
    except Exception as e:
        print(e)


for link in links:
    if n_links % 50 == 0:
        np.savetxt("./texts/addition_700.csv", rev_results, fmt="%s", delimiter=",")
    iteration = 1
    while(True):
        url = link + tail + str(iteration)
        try:
            r = requests.get(url,timeout=5)
            if r.status_code != 200:
                break
            iteration += 1
            review_soup = BeautifulSoup(r.text, 'html.parser')
            review_urls =  [item.get('href') for item in review_soup.find_all('a', class_='rvw-simple-item__title-target')]
            rev_results.append(
                Parallel(n_jobs=4,verbose=10)([
                    delayed(get_review)(i) for i in review_urls
                ])
            )
        except Exception as e:
            print(e)
    

700


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    5.9s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.7s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    9.9s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   14.3s
[Parallel(n_jobs=4)]: Done  63 out of  63 | elapsed:   16.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  63 out of  63 | elapsed:   16.3s finished


701


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    5.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   11.3s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   13.9s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   16.8s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:   18.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:   18.1s finished


702


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.0s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    9.7s
[Parallel(n_jobs=4)]: Done  45 out of  47 | elapsed:   12.3s remaining:    0.5s
[Parallel(n_jobs=4)]: Done  47 out of  47 | elapsed:   12.8s finished


998


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    5.4s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    9.4s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   11.2s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   12.9s finished


999


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    8.3s
[Parallel(n_jobs=4)]: Done  45 out of  47 | elapsed:   11.0s remaining:    0.5s
[Parallel(n_jobs=4)]: Done  47 out of  47 | elapsed:   11.7s finished


In [9]:
# レストラン名と評価のDict作成
# restaurant_names = [item.string for item in html_soup.find_all('a', class_='list-rst__rst-name-target cpy-rst-name')]
# ratings = [item.string for item in html_soup.find_all('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val')]
# for k,v in zip(restaurant_names, ratings):
#     print(k,v)

In [2]:
# f = open('./texts/', 'wb')
# pickle.dump(rev_results, f)
# f.close()

# ロード時コマンド
# results = pickle.load(open('./texts/sample_steak_corpus.csv', 'rb'))

csv_file = open("./texts/restaurant_reviews.csv", "r", errors="", newline="" )

#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

results= [row for row in f]

In [3]:
csv_file = open("./texts/addition_700.csv", "r", errors="", newline="" )
#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

results2= [row for row in f]

In [4]:
rev_results = results + results2[1:]

In [5]:
len(rev_results)

1336

In [7]:
# def mecab_analysis(texts):
#     t = mc.Tagger("-Ochasen")
#     # 辞書ロードうまくいかない
#     #t = mc.Tagger( '-Owakati -d /Users/tominagaryota/mecab-ipadic-neologd')
#     t.parse('')
#     output = []
#     for text in texts:
#             node =  t.parseToNode(text)  
#             while node:
#                 if node.surface != "":  # ヘッダとフッタを除外
#                     word_type = node.feature.split(",")[0]
#                     if word_type in ['名詞', '形容詞']:
#                         output.append(node.surface)
#                     #output.append(node.surface)
#                 node = node.next
#                 if node is None:
#                     break
#     return output

def mecab_analysis(texts):
    t = mc.Tagger("-Ochasen")
    # 辞書ロードうまくいかない
    #t = mc.Tagger( '-Owakati -d /Users/tominagaryota/mecab-ipadic-neologd')
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', '形容詞']:
                output.append(node.surface)
            #output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [ ]:
# sklearnでのLDA。使わないかも

# import mglearn
# from sklearn.decomposition import LatentDirichletAllocation

# vect = CountVectorizer(max_features=10000, max_df=.15)
# # mecab_analysisを使用
# X = vect.fit_transform(mecab_analysis([r[0] for r in rev_results[0]]))
# lda = LatentDirichletAllocation(n_topics=5, learning_method="batch", max_iter=25, random_state=0)
# document_topics = lda.fit_transform(X)

# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())
# mglearn.tools.print_topics(topics=range(5), feature_names=feature_names, 
#                            sorting=sorting, topics_per_chunk=1, n_words=5)

In [ ]:
# vect = CountVectorizer(max_features=10000, max_df=.15)
# N_TOPICS = 6
# # mecab_analysisを使用
# X = vect.fit_transform(mecab_analysis([r[0] for r in rev_results[1]]))
# lda = LatentDirichletAllocation(n_topics=N_TOPICS, learning_method="batch", max_iter=25, random_state=0)
# document_topics = lda.fit_transform(X)

# sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# feature_names = np.array(vect.get_feature_names())
# mglearn.tools.print_topics(topics=range(N_TOPICS), feature_names=feature_names, 
#                            sorting=sorting, topics_per_chunk=1, n_words=10)

In [8]:
all_reviews = []
for rvws in rev_results:
    for r in rvws:
        all_reviews.append(r)

In [9]:
len(all_reviews)

117619

In [84]:
csv_file = open("./texts/wakati.csv", "r", errors="", newline="" )

#リスト形式
f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)

wakati_text = [row for row in f]

In [85]:
new_wakati = []
for text in wakati_text:
    striped =  [t.strip("['n'").strip("'n']").strip("'") for t in text]
    new_wakati.append(striped)
wakati_text = new_wakati

In [87]:
model = word2vec.Word2Vec(new_wakati, size=1000,min_count=5,window=5,iter=100,workers=4,sg=1)

KeyboardInterrupt: 

In [ ]:
model.most_similar('ステーキ')

In [255]:
# モデルのセーブ
model.save("./model/sample_steak_model.model")

In [249]:
#np.savetxt("./texts/sample_steak_corpus.csv", wakati_text, fmt="%s", delimiter=",")

In [259]:
model.most_similar(positive=[u"挽肉",u"ハンバーグ"])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('エビフライ', 0.6223527789115906),
 ('肉汁', 0.5919903516769409),
 ('デミグラスソース', 0.5824424624443054),
 ('肉感', 0.5666879415512085),
 ('フライ', 0.5600730180740356),
 ('俵', 0.5347720384597778),
 ('パクリ', 0.5159295797348022),
 ('デミ', 0.5084888339042664),
 ('ジュワー', 0.5022367238998413),
 ('箸', 0.49991917610168457)]

In [88]:
dictionary = corpora.Dictionary(wakati_text)
dictionary.save_as_text('./texts/text.dict')

In [89]:
corpus = [dictionary.doc2bow(doc) for doc in wakati_text]
corpora.MmCorpus.serialize('./texts/text.mm', corpus)

In [90]:
tfidf = gensim.models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [91]:
with open('./model/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

In [93]:
tfidf_lda = gensim.models.LdaMulticore(corpus=corpus_tfidf, id2word=dictionary,
                             num_topics=10, minimum_probability=0.001,
                             passes=20, eval_every=0, chunksize=1000,workers=4)

In [94]:
tfidf_lda.save('./model/tfidf_lda.model')

In [24]:
# with open('model/tfidf_lda.model', mode='rb') as f:
#     tfidf_lda = pickle.load(f)
tfidf_lda = gensim.models.LdaModel.load('./model/tfidf_lda.model')

In [95]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, tfidf_lda.print_topic(i)[0:80]+'...'))

tpc_0: 0.006*"希少" + 0.004*"おにぎり" + 0.004*"松阪" + 0.003*"テキ" + 0.003*"浜松" + 0.003*"表参道" +...
tpc_1: 0.009*"u3" + 0.008*"000" + 0.003*"お肉" + 0.003*"の" + 0.003*"０" + 0.003*"店" + 0.00...
tpc_2: 0.011*"000" + 0.011*"1" + 0.007*"500円" + 0.007*"u3" + 0.007*"円" + 0.006*"2" + 0....
tpc_3: 0.016*"" + 0.006*"うし" + 0.004*"バンビーナ" + 0.003*"フライデーズ" + 0.003*"極み" + 0.003*"割烹"...
tpc_4: 0.010*"ハンバーグ" + 0.006*"ステーキ" + 0.004*"０" + 0.004*"肉" + 0.004*"ソース" + 0.004*"ライス"...
tpc_5: 0.012*"xa" + 0.010*"トンテキ" + 0.009*"0" + 0.007*"050" + 0.006*"スペアリブ" + 0.003*"108...
tpc_6: 0.012*"牛タン" + 0.004*"焼肉店" + 0.003*"西麻布" + 0.003*"卵黄" + 0.002*"網焼き" + 0.002*"センス"...
tpc_7: 0.005*"280円" + 0.005*"ミスジ" + 0.003*"トニーローマ" + 0.003*"ビアホール" + 0.003*"白子" + 0.003...
tpc_8: 0.006*"げんこつハンバーグ" + 0.005*"ラム" + 0.004*"海鮮" + 0.003*"新橋" + 0.003*"開放" + 0.003*"ハ...
tpc_9: 0.003*"レバ刺し" + 0.003*"ヘレ" + 0.002*"フランス料理" + 0.002*"ローストチキン" + 0.002*"伊勢" + 0.00...


In [352]:
# TF-IDF処理をしないLDA
# raw_lda = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary,
#                              num_topics=10, minimum_probability=0.001,
#                              passes=20, eval_every=0, chunksize=10000,workers=4)

In [44]:
temp ='天下茶屋でランチの機会があればその都度狙っていたこちらの超人気洋食店。売りは一頭買いするらしい黒毛を使ったお肉料理という触れ込みですわ。中でも黒バーグいうハンバーグがメインらしい情報は温めてました＾＾場所は天下茶屋駅から数分です。立ち飲み店がしのぎを削るサイドではない場所でテニススクールのあるあたり。健全でよろしおす(笑)。いうても、平日のランチなんで・・・とはいっても、一番安いハンバーグのランチでも1500円となかなかの高額です。それ相応のレベルは求められるはず。この日は1時前でしたが満席でした。かなり評判がいいみたいです。□黒バーグとカキフライ（1800円）○　エビフライなどの海鮮のフライも人気とのことで、カキフライを＾＾ｖ　まず、肝心の黒バーグですが、うん、僕は美味しいと思いました。　特徴的はなのは、スパイス含め、お肉に味がしっかりついてることと、　牛肉の香りも歯ごたえもあるけど、全然しつこくなくて、なおかつ、　デミグラスソースもあっさりしていて食べやすいです。脂こってりの　濃厚ソースみたいなハンバーグが好きな人には向かないけど、ただ、　　そうだとして、焼きも含めてくいしんぼー山中さんクラスのハンバーグが　といえばあまりに普通かなあ。デミグラスソースも2000円出すにしては。　カキフライは、大きさ重視です。タルタルはおいしいけど、揚げる牡蠣と　してはどうなんだろういう印象。このあたりが、叩き上げのシェフの洋食店　とは一線を画するといったところなのかな。　コンソメ・ご飯は普通にいけました＾＾確かに、この地域で見れば突出してるかもしれないですけど、作法・値段は大阪市内水準です。美味しいのは美味しいんですけどね＾＾ｖ昼に天下茶屋に行かれた際には是非。ご参考まで♪'

In [98]:
vec = dictionary.doc2bow(mecab_analysis(temp))
tfidf_lda[vec]

[(0, 0.016632209),
 (1, 0.46797404),
 (4, 0.48619828),
 (6, 0.0087923305),
 (8, 0.016394185)]

In [31]:
# LDA分類したベクトルをnumpyベクトル化
def convert_to_np_vec(vec):
    n_v = np.zeros(50)
    for v in vec:
        n_v[v[0]] = v[1]
    return n_v

In [32]:
# コサイン類似度を計算
def cos_sim(v1,v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [99]:
vec = dictionary.doc2bow(mecab_analysis(rvws[10]))
vec2 = dictionary.doc2bow(mecab_analysis(rvws[14]))
vec1 = convert_to_np_vec(tfidf_lda[vec])
vec2 = convert_to_np_vec(tfidf_lda[vec2])

In [101]:
cos_sim(vec1,vec2)

0.7385122087001152

In [102]:
data = word2vec.Text8Corpus("./wiki/wiki_wakati.txt")
model =  word2vec.Word2Vec(data, size=100)

FileNotFoundError: [Errno 2] No such file or directory: ',/model/wiki.model.wv.vectors.npy'

In [104]:
model.most_similar(positive = ['純粋','悪'],negative = ['正義'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('単純', 0.7389036417007446),
 ('低級', 0.7317520380020142),
 ('邪悪', 0.6965624690055847),
 ('不完全', 0.6847642660140991),
 ('無機質', 0.6803640127182007),
 ('鋭敏', 0.6790430545806885),
 ('複雑', 0.6772918701171875),
 ('微細', 0.676936686038971),
 ('獰猛', 0.6749590635299683),
 ('醜悪', 0.6736862063407898)]

In [109]:
model.save('./model/wiki.model')